In [a blog post by Darrow Kirkpatrick][2] he uses CAPE10 to decide whether to rebalance from stocks or bonds.

Let's try to replicate that.

* $1,000,000 portfolio
* 4% initial withdrawal rate
* Inflation-adjusted
* 30-year retirement
* 50/50 between S&P 500 and 10-year treasuries
* Make a withdrawal at the beginning of the year
* Perform annual rebalancing as well? Unclear. His [first time][1] he doesn't appear to ever do rebalancing.
  In his [newer one][2] he lists rebalancing options but somehow(?) they provide different results. I don't
  understand how that is possible. If you end up rebalancing after taking the withdrawal, then it doesn't matter
  where you take the withdrawal from. Unless he rebalances **first** and then **then** takes the withdrawal?

[1]: http://www.caniretireyet.com/new-research-the-best-retirement-withdrawal-strategies/
[2]: http://www.caniretireyet.com/the-best-retirement-withdrawal-strategies-digging-deeper/

In [1]:
import pandas
import portfolio
import collections
from decimal import Decimal
from adt import AnnualChange

In [2]:
Date = collections.namedtuple('Date', ['year', 'month'])

def str_to_date(s):
    (year, month) = s.split('.')
    def convert_month(m):
        if m == '01':
            return 1
        elif m == '1':
            return 10
        else:
            return int(m)
    return Date(int(year), convert_month(month))

frame = pandas.read_csv('schiller_monthly.csv', converters={'Date': str_to_date})

CAPE_STARTS = Date(1881, 1)
DATA_STARTS = Date(1871, 1)
MONTHS_IN_YEAR = 12
YEARS_IN_RETIREMENT = 30
LENGTH_OF_RETIRMENT = YEARS_IN_RETIREMENT * MONTHS_IN_YEAR

def get_row(date):
    years = date.year - DATA_STARTS.year
    months = years * MONTHS_IN_YEAR
    months += date.month - DATA_STARTS.month
    return months

assert frame.iloc[get_row(CAPE_STARTS)]['Date'] == CAPE_STARTS

In [3]:
def retire(frame, start_date):
    assert start_date.year >= CAPE_STARTS.year
    
    current_date = start_date
    
    p = portfolio.Portfolio(500000, 500000)
    last_equity_index = frame.iloc[get_row(start_date)]['S&P Price']

    for i in range(LENGTH_OF_RETIRMENT):
        df = frame.iloc[get_row(current_date)]
        
        # update portfolio
        current_equity_index = frame.iloc[get_row(current_date)]['S&P Price']
        percent_change = (current_equity_index / last_equity_index) - 1
        last_equity_index = current_equity_index
        
        dividends = frame.iloc[get_row(current_date)]['S&P Dividend']
        monthly_yield = (dividends / current_equity_index) / 12
        
        dollar_change = (p.stocks * Decimal(percent_change)) + (p.stocks * Decimal(monthly_yield))

        stock_change = Decimal(percent_change) + Decimal(monthly_yield)

        p.adjust_returns(AnnualChange(year=0, stocks=stock_change, bonds=0, inflation=0))

        if current_date.month == start_date.month:
            print(df['Date'], p.value)
            # make withdrawal based on CAPE
            # rebalance?
            
        new_date_months = current_date.year * MONTHS_IN_YEAR + current_date.month
        current_date = Date(new_date_months // 12, (new_date_months % 12) + 1)

#retire(frame, Date(1900, 1))

In [18]:
def make_csv(frame):
    pd = pandas.DataFrame(columns=['CAPE10', 'Mean', 'Median'])
    for row, i in frame.iloc[0:].iterrows():
        if i['Date'].month == 12 and i['Date'].year >= 1881:
            pd.loc[i['Date'].year] = {'CAPE10': i['CAPE10'], 'Mean': i['Mean'], 'Median': i['Median']}
    pd.to_csv('cape10.csv')
    print(pd.head())
make_csv(frame)

      CAPE10   Mean  Median
1881   15.96  17.63   18.03
1882   15.38  16.44   16.02
1883   14.90  16.06   15.51
1884   13.43  15.49   15.26
1885   16.30  15.31   15.17
